In [34]:
import bs4
import csv
import datetime
import urllib.request
import json
import pymysql

VISIT_BUSAN_URL = ("https://www.visitbusan.net/index.do?menuCd=DOM_000000201001000000")

html_obj = urllib.request.urlopen(VISIT_BUSAN_URL)
web_page = html_obj.read()
bs_obj = bs4.BeautifulSoup(web_page, 'html.parser')

contents = bs_obj.find_all('div', {'class': 'hot-item'})

def get_content_info(content):
    thumbnail = content.find('img')["alt"]
    click_count = content.find("img", alt="클릭수").next_sibling.strip()
    like_count = content.find("img", alt="좋아요").next_sibling.strip()
    return [thumbnail, click_count, like_count]



In [35]:
now = datetime.datetime.now()
yymmdd = now.strftime('%Y%m%d')

csvfile = f'visit_busan_{yymmdd}.csv'

result_list = []
for content in contents:
    result_list.append(get_content_info(content))

with open(csvfile, 'a', newline='') as csvfile:
    fieldnames = ['thumbnail', 'click_count', 'like_count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    for result in result_list:
        writer.writerow(dict(zip(fieldnames, result)))

In [36]:
with open(f'visit_busan_{yymmdd}.json', 'w', encoding='utf-8') as outfile:
    jsonFile = json.dumps(result_list, indent=4, sort_keys=True, ensure_ascii=False)
    outfile.write(jsonFile)

In [37]:
db = pymysql.connect(host="localhost", user="webuser", passwd="webuser", db="webdb")

cursor = db.cursor()

query = "INSERT INTO busanVisit (thumbnail, click_count, like_count) VALUES (%s, %s, %s)"

try:
    cursor.executemany(query, result_list)
    db.commit()
    print(f"DB 저장 완료")
except Exception as e:
    db.rollback()
    print(f"DB 저장 실패 : {e}")
cursor.close()
db.close()

DB 저장 완료
